# K-Means Implementation: From Scratch to Production

In this notebook, we'll:

1. Implement K-Means from scratch using pure NumPy
2. Compare with scikit-learn's implementation
3. Understand K-Means++ initialization
4. Benchmark performance and accuracy
5. Apply elbow method and silhouette analysis

Let's build K-Means step by step!

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans as SklearnKMeans
from sklearn.metrics import silhouette_score
import time
import sys
sys.path.append('../src')

# Import our custom implementation
from kmeans import KMeans
from utils import plot_clusters_2d, plot_elbow, plot_silhouette_scores

import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')

print("✅ Libraries imported successfully!")

## 1. Create Sample Data

Let's generate synthetic data with known clusters for testing:

In [ ]:
# Generate synthetic data
np.random.seed(42)
X, y_true = make_blobs(
    n_samples=500,
    centers=4,
    n_features=2,
    cluster_std=0.8,
    random_state=42
)

print(f"Dataset shape: {X.shape}")
print(f"Number of samples: {X.shape[0]}")
print(f"Number of features: {X.shape[1]}")

# Visualize the raw data
plt.figure(figsize=(10, 6))
plt.scatter(X[:, 0], X[:, 1], s=50, alpha=0.6, edgecolors='black', linewidths=0.5)
plt.title('Unlabeled Data', fontsize=14, fontweight='bold')
plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Our From-Scratch Implementation

We've already implemented a complete K-Means class in `src/kmeans.py`. Let's test it!

### Key Features of Our Implementation:
- **K-Means++ initialization:** Smart centroid selection for better convergence
- **Convergence detection:** Stops when centroids stabilize
- **Inertia tracking:** Measures cluster quality (WCSS)
- **Pure NumPy:** No external dependencies (except NumPy)

In [ ]:
# Initialize our custom K-Means
kmeans_custom = KMeans(
    n_clusters=4,
    max_iters=300,
    init='kmeans++',
    random_state=42
)

# Fit the model
start_time = time.time()
kmeans_custom.fit(X)
custom_time = time.time() - start_time

print(f"✅ Custom K-Means completed!")
print(f"   Number of iterations: {kmeans_custom.n_iter_}")
print(f"   Final inertia (WCSS): {kmeans_custom.inertia_:.2f}")
print(f"   Execution time: {custom_time*1000:.2f} ms")
print(f"   Cluster centers shape: {kmeans_custom.cluster_centers_.shape}")

In [ ]:
# Visualize results with Plotly
fig = plot_clusters_2d(
    X,
    kmeans_custom.labels_,
    kmeans_custom.cluster_centers_,
    title="Custom K-Means Implementation Results",
    feature_names=['Feature 1', 'Feature 2']
)
fig.show()

print("🎨 Interactive plot: Hover to see details, zoom, pan, and export!")

## 3. Compare with scikit-learn

Let's benchmark our implementation against the production-ready sklearn version:

In [ ]:
# Initialize sklearn K-Means
kmeans_sklearn = SklearnKMeans(
    n_clusters=4,
    init='k-means++',
    max_iter=300,
    n_init=1,  # Single run for fair comparison
    random_state=42
)

# Fit the model
start_time = time.time()
kmeans_sklearn.fit(X)
sklearn_time = time.time() - start_time

print(f"✅ sklearn K-Means completed!")
print(f"   Number of iterations: {kmeans_sklearn.n_iter_}")
print(f"   Final inertia (WCSS): {kmeans_sklearn.inertia_:.2f}")
print(f"   Execution time: {sklearn_time*1000:.2f} ms")
print(f"   Cluster centers shape: {kmeans_sklearn.cluster_centers_.shape}")

In [ ]:
# Side-by-side comparison
comparison_df = pd.DataFrame({
    'Metric': ['Inertia (WCSS)', 'Iterations', 'Time (ms)', 'Time Ratio'],
    'Custom Implementation': [
        f"{kmeans_custom.inertia_:.2f}",
        kmeans_custom.n_iter_,
        f"{custom_time*1000:.2f}",
        "1.0x (baseline)"
    ],
    'scikit-learn': [
        f"{kmeans_sklearn.inertia_:.2f}",
        kmeans_sklearn.n_iter_,
        f"{sklearn_time*1000:.2f}",
        f"{sklearn_time/custom_time:.2f}x"
    ]
})

print("\n📊 Performance Comparison:")
print(comparison_df.to_string(index=False))
print("\n💡 Note: sklearn is typically faster due to optimized C implementations.")

In [ ]:
# Visualize both results side by side
from utils import create_comparison_figure

labels_dict = {
    'Custom K-Means': kmeans_custom.labels_,
    'sklearn K-Means': kmeans_sklearn.labels_
}

centroids_dict = {
    'Custom K-Means': kmeans_custom.cluster_centers_,
    'sklearn K-Means': kmeans_sklearn.cluster_centers_
}

fig = create_comparison_figure(
    X,
    labels_dict,
    centroids_dict,
    title="Custom vs sklearn K-Means Comparison"
)
fig.show()

print("Both implementations produce nearly identical results! ✨")

## 4. K-Means++ vs Random Initialization

**K-Means++** is a smart initialization method that:
1. Chooses first centroid randomly
2. For each subsequent centroid, chooses points far from existing centroids (with probability proportional to distance²)

**Why it matters:** Better initialization → faster convergence, better final clusters

Let's compare random vs K-Means++ initialization:

In [ ]:
# Random initialization
kmeans_random = KMeans(n_clusters=4, init='random', random_state=42)
kmeans_random.fit(X)

# K-Means++ initialization
kmeans_plusplus = KMeans(n_clusters=4, init='kmeans++', random_state=42)
kmeans_plusplus.fit(X)

print("Comparison: Random vs K-Means++ Initialization\n")
print(f"Random Init:")
print(f"  Iterations: {kmeans_random.n_iter_}")
print(f"  Final inertia: {kmeans_random.inertia_:.2f}\n")

print(f"K-Means++ Init:")
print(f"  Iterations: {kmeans_plusplus.n_iter_}")
print(f"  Final inertia: {kmeans_plusplus.inertia_:.2f}\n")

improvement = (1 - kmeans_plusplus.inertia_ / kmeans_random.inertia_) * 100
print(f"📈 K-Means++ achieved {improvement:.1f}% better clustering (lower inertia)")
print(f"⚡ K-Means++ converged in {kmeans_random.n_iter_ - kmeans_plusplus.n_iter_} fewer iterations")

In [ ]:
# Visualize the difference
labels_dict_init = {
    'Random Initialization': kmeans_random.labels_,
    'K-Means++ Initialization': kmeans_plusplus.labels_
}

centroids_dict_init = {
    'Random Initialization': kmeans_random.cluster_centers_,
    'K-Means++ Initialization': kmeans_plusplus.cluster_centers_
}

fig = create_comparison_figure(
    X,
    labels_dict_init,
    centroids_dict_init,
    title="Impact of Initialization Method"
)
fig.show()

## 5. Elbow Method Implementation

Now let's use our implementation to find the optimal K:

In [ ]:
# Test different values of K
K_range = range(1, 11)
inertias = []

print("Running K-Means for different K values...")
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)
    print(f"  K={k}: inertia={kmeans.inertia_:.2f}")

print("\n✅ Elbow analysis complete!")

In [ ]:
# Plot the elbow curve using our utility function
fig = plot_elbow(K_range, inertias, title="Elbow Method: Finding Optimal K")
fig.show()

print("📍 Look for the 'elbow' where the curve bends sharply.")
print("Based on the plot, K=4 appears to be optimal!")

## 6. Silhouette Analysis

Let's complement the elbow method with silhouette scores:

In [ ]:
# Calculate silhouette scores
K_range_sil = range(2, 11)  # Need at least 2 clusters for silhouette
silhouette_scores = []

print("Calculating silhouette scores...")
for k in K_range_sil:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X)
    score = silhouette_score(X, labels)
    silhouette_scores.append(score)
    print(f"  K={k}: silhouette score={score:.3f}")

print("\n✅ Silhouette analysis complete!")

In [ ]:
# Plot silhouette scores
fig = plot_silhouette_scores(
    K_range_sil,
    silhouette_scores,
    title="Silhouette Score Analysis"
)
fig.show()

best_k_idx = np.argmax(silhouette_scores)
best_k = list(K_range_sil)[best_k_idx]
print(f"\n🏆 Best K according to silhouette score: {best_k}")
print(f"Silhouette score: {silhouette_scores[best_k_idx]:.3f}")

In [ ]:
# Detailed silhouette analysis for K=4
from utils import plot_silhouette_analysis

kmeans_final = KMeans(n_clusters=4, random_state=42)
labels_final = kmeans_final.fit_predict(X)

fig = plot_silhouette_analysis(X, labels_final, n_clusters=4)
plt.show()

print("\n📊 Silhouette plot interpretation:")
print("  • Wider bars = better-defined clusters")
print("  • Values close to 1 = well-clustered points")
print("  • Values near 0 or negative = points on cluster boundaries or misclassified")

## 7. Prediction on New Data

Once we've fit K-Means, we can assign new points to existing clusters:

In [ ]:
# Generate new test points
X_new = np.array([
    [0, 0],      # Point near cluster 1
    [10, 10],    # Point near cluster 2
    [-8, -8],    # Point near cluster 3
    [5, -8]      # Point near cluster 4
])

# Predict cluster assignments
predictions = kmeans_final.predict(X_new)

print("New points and their predicted clusters:\n")
for i, (point, cluster) in enumerate(zip(X_new, predictions)):
    print(f"  Point {i+1}: {point} → Cluster {cluster}")

In [ ]:
# Visualize predictions
fig = plot_clusters_2d(
    X,
    labels_final,
    kmeans_final.cluster_centers_,
    title="K-Means with New Point Predictions",
    feature_names=['Feature 1', 'Feature 2']
)

# Add new points
colors_new = px.colors.qualitative.Set2
for i, (point, cluster) in enumerate(zip(X_new, predictions)):
    fig.add_trace(go.Scatter(
        x=[point[0]],
        y=[point[1]],
        mode='markers',
        marker=dict(
            size=20,
            color=colors_new[cluster],
            symbol='star',
            line=dict(width=2, color='black')
        ),
        name=f'New Point {i+1} (Cluster {cluster})',
        showlegend=True
    ))

fig.show()

print("⭐ New points shown as stars, colored by their predicted cluster")

## Summary

In this notebook, we:

1. ✅ **Implemented K-Means from scratch** in pure NumPy with K-Means++ initialization
2. ✅ **Compared with sklearn** and found nearly identical results
3. ✅ **Demonstrated K-Means++** superiority over random initialization
4. ✅ **Applied the elbow method** to find optimal K
5. ✅ **Used silhouette analysis** to validate cluster quality
6. ✅ **Made predictions** on new data points

### Key Takeaways

- **K-Means++ initialization** is essential for good results
- **Combine multiple methods** (elbow + silhouette) to choose K
- **Our implementation** is correct but sklearn is faster (optimized C code)
- **Silhouette scores** provide quantitative cluster quality assessment

### Next Steps

In **Notebook 03**, we'll apply K-Means to real-world problems:
- Customer segmentation for targeted marketing
- Image compression using color clustering
- Business insights and actionable recommendations

---

**Ready for real data?** → [03_real_world.ipynb](03_real_world.ipynb)